In [7]:

import pandas as pd
import numpy as np
import os   



In [8]:
data_folder = "./data/"
clean_data = os.path.join(data_folder, "clean_data.csv")

In [9]:
data = pd.read_csv(clean_data, index_col=0 , header=[0, 1], parse_dates=True, dtype='float64')
data.head() 


Location               Tank 1                                              \
Custom Name           O2 perc pH regulator in flow Sludge recycle in flow   
2022-01-17 17:10:00  18.29834             27.83947               99.93546   
2022-01-17 17:15:00  18.29748             27.67788               99.93343   
2022-01-17 17:20:00  18.29662             27.51629               99.93140   
2022-01-17 17:25:00  18.29576             27.35470               99.92937   
2022-01-17 17:30:00  18.29490             27.19311               99.92734   

Location                                                               \
Custom Name         Air in flow Content height        pH O2 dissolved   
2022-01-17 17:10:00    747.0399            NaN  7.298596     1.609646   
2022-01-17 17:15:00    744.1708            NaN  7.298001     1.607517   
2022-01-17 17:20:00    741.3018            NaN  7.297405     1.605388   
2022-01-17 17:25:00    738.4326            NaN  7.296809     1.603259   
2022-01-17 17:30:00    735.5635            NaN  7.296214     1.601130   

Location                                        Exit               ...  \
Custom Name         Temperature Sludge concentration Conductivity  ...   
2022-01-17 17:10:00    26.66654             1.502616     35.95628  ...   
2022-01-17 17:15:00    26.66875             1.504003     35.88463  ...   
2022-01-17 17:20:00    26.67095             1.505389     35.81298  ...   
2022-01-17 17:25:00    26.67315             1.506775     35.74133  ...   
2022-01-17 17:30:00    26.67536             1.508161     35.66968  ...   

Location                 Input A                         Input B            \
Custom Name         Conductivity       TOC        pH Temperature  Flowrate   
2022-01-17 17:10:00     51.19062  14.25161  6.559590    24.44635  29.16244   
2022-01-17 17:15:00     51.17931  14.18093  6.424222    24.59840  17.46023   
2022-01-17 17:20:00     51.16800  14.11025  6.634570    24.59937  45.92802   
2022-01-17 17:25:00     51.15670  14.03957  6.639138    24.30975  30.36573   
2022-01-17 17:30:00     51.14539  13.96888  6.662630    24.10653  26.96781   

Location                                                             Exit  
Custom Name         Amount of sludge Conductivity TOC        pH    Target  
2022-01-17 17:10:00         144.6119     2.364562 NaN  6.930508  67.15141  
2022-01-17 17:15:00         144.4974     2.368205 NaN  6.931771  67.14989  
2022-01-17 17:20:00         144.3829     2.371849 NaN  6.933034  67.14836  
2022-01-17 17:25:00         144.2685     2.375492 NaN  6.934297  67.14684  
2022-01-17 17:30:00         144.1540     2.379136 NaN  6.935560  67.14531  

[5 rows x 31 columns]